# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [66]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [67]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.56,95,100,16.19,TF,1698946890
1,1,iqaluit,63.7506,-68.5145,-3.15,63,75,6.17,CA,1698946896
2,2,khmilnyk,49.5598,27.9575,8.99,69,65,4.87,UA,1698946899
3,3,albany,42.6001,-73.9662,8.66,50,21,0.45,US,1698946900
4,4,dryden,49.7833,-92.7503,-0.84,74,100,4.63,CA,1698946903


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [119]:
import geoviews.tile_sources as gvts


pd.options.plotting.backend = "holoviews"

# Assuming you have a DataFrame 'city_data_df' with columns "Lng", "Lat", "Humidity", "City", and "Country"
df = pd.DataFrame(
    city_data_df,
    columns=[
        "Lng",
        "Lat",
        "Humidity",
        "City",
        "Country",  # Assuming you have a "Country" column
    ],
)

tile_options = {"width": 700, "height": 500, "xaxis": None, "yaxis": None}
world = gvts.OSM.opts(**tile_options)

# Calculate alpha values based on humidity
df["alpha"] = df["Humidity"] / 100  # Adjust scaling as needed

world * df.hvplot.points(
    geo=True, s="Humidity", c="City", alpha="alpha", hover_cols=["City", "Humidity"]
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,alpha)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [122]:
# Define your criteria (example criteria provided)
max_temp_criteria = (city_data_df["Max Temp"] > 21) & (
    city_data_df["Max Temp"] < 27)
cloudiness_criteria = city_data_df["Cloudiness"] < 20
wind_speed_criteria = city_data_df["Wind Speed"] < 4

# Combine all criteria to find cities that meet all conditions
filtered_cities = pd.DataFrame.copy(
    city_data_df[max_temp_criteria & cloudiness_criteria & wind_speed_criteria]
)

# Drop any rows with null values
filtered_cities.dropna(inplace=True)

# Display sample data (first 5 rows)
filtered_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,hadibu,12.6500,54.0333,26.80,81,0,4.15,YE,1698946906
20,20,khark,29.2614,50.3306,25.01,78,0,5.14,IR,1698946929
22,22,arraial do cabo,-22.9661,-42.0278,24.98,73,0,4.63,BR,1698946936
25,25,georgetown,5.4112,100.3354,26.96,92,0,1.03,MY,1698946701
35,35,kiunga,-6.1219,141.2906,21.26,99,0,1.03,PG,1698946972


### Step 3: Create a new DataFrame called `hotel_df`.

In [123]:
# Create a copy of the filtered_cities DataFrame and select the columns you need
hotel_df = filtered_cities[[
    "City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,hadibu,YE,12.6500,54.0333,81,
20,khark,IR,29.2614,50.3306,78,
22,arraial do cabo,BR,-22.9661,-42.0278,73,
25,georgetown,MY,5.4112,100.3354,92,
35,kiunga,PG,-6.1219,141.2906,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [137]:
import requests

# Set your Geoapify API key
api_key = geoapify_key

# Set the base URL
base_url = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 kilometers)
categories = "accommodation.hotel"
limit = 1  # Limit to one result (the nearest hotel)

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Construct the API request URL with the specific city's coordinates
    url = base_url + "?categories=" + categories + "&filter=circle:" + str(lng) + "," + str(lat) + "," + str(
        radius) + "&bias=proximity:" + str(lng) + "," + str(lat) + "&limit=" + str(limit) + "&apiKey=" + api_key

    # Make an API request using the constructed URL
    response = requests.get(url)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.at[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.at[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.at[index, 'City']
             } - nearest hotel: {hotel_df.at[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)

Starting hotel search
hadibu - nearest hotel: No hotel found
khark - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
kiunga - nearest hotel: No hotel found
betio village - nearest hotel: Seaman Hostel
micheweni - nearest hotel: No hotel found
bengkulu - nearest hotel: Dena Hotel
essaouira - nearest hotel: Essaouira Wind Palace
fish town - nearest hotel: No hotel found
pagar alam - nearest hotel: Villa Dempo Flower
cochin - nearest hotel: Hotel Top Form
tapah road - nearest hotel: Rumah Rehat Tapah
manokwari - nearest hotel: Hotel Arfak
arteche - nearest hotel: No hotel found
as saffaniyah - nearest hotel: No hotel found
qui nhon - nearest hotel: TMS Quy Nhơn - Home ở đây!
isangel - nearest hotel: Tanna Lodge
katsuren-haebaru - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
kakata - nearest hotel: No hotel found
pangkalanbuun - nearest hotel: Hotel Tiara
candelaria - nearest hotel: Grand Central H

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [144]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE


import geoviews.tile_sources as gvts


pd.options.plotting.backend = "holoviews"

# Assuming you have a DataFrame 'city_data_df' with columns "Lng", "Lat", "Humidity", "City", and "Country"
df = pd.DataFrame(
    hotel_df,
    columns=[
        "Lng",
        "Lat",
        "Humidity",
        "City",
        "Country",
        "Hotel Name"  # Assuming you have a "Country" column
    ],
)

tile_options = {"width": 700, "height": 500, "xaxis": None, "yaxis": None}
world = gvts.OSM.opts(**tile_options)

# Calculate alpha values based on humidity
df["alpha"] = df["Humidity"] / 100  # Adjust scaling as needed

world * df.hvplot.points(
    geo=True, s="Humidity", c="City", alpha="alpha", hover_cols=["City", "Humidity","Hotel Name","Country"]
)


# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,alpha,Hotel Name,Country)